# Order Brushing

## Tasks
1. Identify all shops that are deemed to have conducted order brushing.
2. For each shop that is identified to have conducted order brushing, identify the buyers suspected to have conducted order brushing for that shop.


#### concentration_rate = Number of Orders within 1 hour / Number of Unique Buyers within 1 hour

Shops are deemed to have conducted order brushing if their concentration_rate >= 3 at any instance, and suspicious buyers are deemed as the buyer that contributed the __**highest**__ proportion of orders to a shop that is deemed to have conducted order brushing.

For calculation of the highest proportion of orders to a shop, only include the orders that occured in instances when order brushing has been deemed to have taken place.

## Importing and Data Preprocessing

In [1]:
# Import package(s)
import pandas as pd

# Import Data and Pre-processing
data_name = "order_brush_order.csv"
df = pd.read_csv(data_name)
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 222750 entries, 0 to 222749
Data columns (total 4 columns):
 #   Column      Non-Null Count   Dtype 
---  ------      --------------   ----- 
 0   orderid     222750 non-null  int64 
 1   shopid      222750 non-null  int64 
 2   userid      222750 non-null  int64 
 3   event_time  222750 non-null  object
dtypes: int64(3), object(1)
memory usage: 6.8+ MB


In [2]:
# Ensure all columns are of the correct format
df["orderid"] = df["orderid"].astype(str)
df["shopid"] = df["shopid"].astype(str)
df["userid"] = df["userid"].astype(str)
df["event_time"] = pd.to_datetime(df["event_time"])
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 222750 entries, 0 to 222749
Data columns (total 4 columns):
 #   Column      Non-Null Count   Dtype         
---  ------      --------------   -----         
 0   orderid     222750 non-null  object        
 1   shopid      222750 non-null  object        
 2   userid      222750 non-null  object        
 3   event_time  222750 non-null  datetime64[ns]
dtypes: datetime64[ns](1), object(3)
memory usage: 6.8+ MB


In [3]:
# Let's reorder the columns a bit
df = df.set_index("event_time").sort_index()[["shopid", "userid", "orderid"]]
df

,shopid,userid,orderid
event_time,,,
2019-12-27 00:00:00,6042309,97707522,31075200506751
2019-12-27 00:00:00,104804492,97707522,31075200506752
2019-12-27 00:00:00,8715449,97707522,31075200506753
2019-12-27 00:00:02,190969466,170182475,31075201870570
2019-12-27 00:00:05,2859407,12532131,31075205798264
...,...,...,...
2019-12-31 23:59:43,149254894,193333760,31507183252446
2019-12-31 23:59:47,147941492,40258063,31507187390691
2019-12-31 23:59:51,187123853,2338306,31507191066628


## Rolling window count unique orderids and userids.
### We will do a rolling window of 1 hour.

Groupby shopid and count how many orderids and unique userids are there in each rolling window, which will be used for our concentration rate. However, rolling windows does not usual let us do a nunique().

Refer: [https://stackoverflow.com/questions/46470743/how-to-efficiently-compute-a-rolling-unique-count-in-a-pandas-time-series](https://stackoverflow.com/questions/46470743/how-to-efficiently-compute-a-rolling-unique-count-in-a-pandas-time-series)


In [4]:
# WARNING: This line took some time to run. So after this I will copy onto another df to avoid running this.
df_rolling_unique = df.groupby(["shopid"]).rolling("1h").apply(lambda arr: pd.Series(arr).nunique())
df_rolling_unique

shopid  userid  orderid
shopid    event_time                                  
100028340 2019-12-31 10:32:50     1.0     1.0      1.0
100028604 2019-12-28 18:16:43     1.0     1.0      1.0
100028732 2019-12-27 02:02:04     1.0     1.0      1.0
          2019-12-27 16:13:49     1.0     1.0      1.0
          2019-12-28 15:01:31     1.0     1.0      1.0
...                               ...     ...      ...
99893455  2019-12-31 16:46:34     1.0     1.0      1.0
          2019-12-31 17:32:16     1.0     2.0      2.0
          2019-12-31 18:39:59     1.0     1.0      1.0
          2019-12-31 19:31:30     1.0     2.0      2.0
          2019-12-31 22:07:53     1.0     1.0      1.0

[222750 rows x 3 columns]

In [5]:
# Copy onto another df so we don't have to run it again (took a while more than 5 minutes)
df_rolling_unique2 = df_rolling_unique.copy()

# Some cleaning
df_rolling_unique2 = df_rolling_unique2.drop("shopid", axis = 1)
df_rolling_unique2 = df_rolling_unique2.rename(columns = {"userid": "unique_user_counts",
                                                          "orderid" : "order_counts"})
df_rolling_unique2 = df_rolling_unique2.reset_index()
df_rolling_unique2

,shopid,event_time,unique_user_counts,order_counts
0,100028340,2019-12-31 10:32:50,1.0,1.0
1,100028604,2019-12-28 18:16:43,1.0,1.0
2,100028732,2019-12-27 02:02:04,1.0,1.0
3,100028732,2019-12-27 16:13:49,1.0,1.0
4,100028732,2019-12-28 15:01:31,1.0,1.0
...,...,...,...,...
222745,99893455,2019-12-31 16:46:34,1.0,1.0
222746,99893455,2019-12-31 17:32:16,2.0,2.0
222747,99893455,2019-12-31 18:39:59,1.0,1.0
222748,99893455,2019-12-31 19:31:30,2.0,2.0


## Now we find out concentration_rate and isolate those shopids with order brushing

In [6]:
# Concentration rate for that rolling window
df_rolling_unique2["concentration_rate"] = df_rolling_unique2["order_counts"] / df_rolling_unique2["unique_user_counts"] 

# If concentration_rate >= 3 then should be order brushed (ob). For easier identification.
df_rolling_unique2["ob"] = df_rolling_unique2["concentration_rate"] >= 3
df_rolling_unique2

,shopid,event_time,unique_user_counts,order_counts,concentration_rate,ob
0,100028340,2019-12-31 10:32:50,1.0,1.0,1.0,False
1,100028604,2019-12-28 18:16:43,1.0,1.0,1.0,False
2,100028732,2019-12-27 02:02:04,1.0,1.0,1.0,False
3,100028732,2019-12-27 16:13:49,1.0,1.0,1.0,False
4,100028732,2019-12-28 15:01:31,1.0,1.0,1.0,False
...,...,...,...,...,...,...
222745,99893455,2019-12-31 16:46:34,1.0,1.0,1.0,False
222746,99893455,2019-12-31 17:32:16,2.0,2.0,1.0,False
222747,99893455,2019-12-31 18:39:59,1.0,1.0,1.0,False
222748,99893455,2019-12-31 19:31:30,2.0,2.0,1.0,False


In [7]:
# Isolate all the order brushing orders! (ob = True)
df_ob_shopid = df_rolling_unique2[df_rolling_unique2["ob"]]
df_ob_shopid

,shopid,event_time,unique_user_counts,order_counts,concentration_rate,ob
266,100446829,2019-12-28 10:20:45,1.0,3.0,3.0,True
267,100446829,2019-12-28 10:23:12,1.0,4.0,4.0,True
2241,10199219,2019-12-31 14:28:53,1.0,3.0,3.0,True
2368,10206302,2019-12-29 01:16:36,1.0,3.0,3.0,True
2369,10206302,2019-12-29 01:17:33,1.0,4.0,4.0,True
...,...,...,...,...,...,...
220479,98481320,2019-12-29 23:33:01,1.0,3.0,3.0,True
220487,98481320,2019-12-31 23:31:26,2.0,6.0,3.0,True
220489,98481320,2019-12-31 23:40:05,2.0,7.0,3.5,True
221336,98793086,2019-12-30 23:00:46,1.0,3.0,3.0,True


## Find all order brushed shopid-userid

In [8]:
# List of all shopid that have order brushed.
# Note: This is not a dataframe, but a list.
ob_shopid = df_ob_shopid["shopid"].drop_duplicates()
ob_shopid
# 264 shopids with order brushing

266       100446829
2241       10199219
2368       10206302
4142      103715156
4474          10402
            ...    
216896     96555082
219341     97167904
220479     98481320
221336     98793086
221703     99067259
Name: shopid, Length: 264, dtype: object

In [9]:
# Give me all orders in the main dataframe by shopids that have order brushed
df_ob = df[df["shopid"].isin(ob_shopid)]
df_ob

,shopid,userid,orderid
event_time,,,
2019-12-27 00:00:18,68862371,450199,31075218789779
2019-12-27 00:00:54,27476241,31032297,31075254760568
2019-12-27 00:00:55,192608876,3089383,31075255453387
2019-12-27 00:02:30,10402,185138818,31075350383695
2019-12-27 00:02:59,114919380,101404887,31075378210184
...,...,...,...
2019-12-31 23:58:32,126587078,84577879,31507111667025
2019-12-31 23:58:49,43412276,12417681,31507128830714
2019-12-31 23:59:26,198172755,115112195,31507166937011


## Pairing (Groupby) shopid and userid, find out how many orders each pair did (e.g. unique orderid per pair)

In [10]:
df_ob_counts = df_ob.groupby(["shopid", "userid"]).nunique()
df_ob_counts

shopid  userid  orderid
shopid    userid                            
100446829 2434757         1       1        4
10199219  47314841        1       1        2
          8405753         1       1        3
10206302  101137867       1       1        1
          103382549       1       1        1
...                     ...     ...      ...
99067259  38908587        1       1        1
          71979496        1       1        1
          95958383        1       1        1
          9670331         1       1        1
          9996458         1       1        1

[8260 rows x 3 columns]

In [11]:
# Some cleaning up helps
df_ob_counts = df_ob_counts.drop(["shopid", "userid"], axis = 1).reset_index()
df_ob_counts = df_ob_counts.rename(columns = {"orderid" : "order_counts"})
df_ob_counts

,shopid,userid,order_counts
0,100446829,2434757,4
1,10199219,47314841,2
2,10199219,8405753,3
3,10206302,101137867,1
4,10206302,103382549,1
...,...,...,...
8255,99067259,38908587,1
8256,99067259,71979496,1
8257,99067259,95958383,1
8258,99067259,9670331,1


### We now have
* All shopids that have order brushed;
* All userids that helped each shopid to brush; and
* The number of times each userid helped to brush each shopid.

## Now for each shopid, we need to find the userid(s) brushed the MOST (max order_counts) and remove all non-max userids.

Refer: [https://stackoverflow.com/questions/15705630/get-the-rows-which-have-the-max-count-in-groups-using-groupby](https://stackoverflow.com/questions/15705630/get-the-rows-which-have-the-max-count-in-groups-using-groupby)
<br>
*Note: If you have multiple max values per group, all will be returned.*
<br>
#### This is what we want, if there are multiple max entries then we want to append them into the final df.

In [12]:
# Get index of all the rows that have max order_counts for each shopid
idx = df_ob_counts.groupby(["shopid"], sort = False)["order_counts"].transform(max) == df_ob_counts["order_counts"]
df_ob_max = df_ob_counts[idx]
df_ob_max
# 286 userids with max order_counts

,shopid,userid,order_counts
0,100446829,2434757,4
2,10199219,8405753,3
41,10206302,95058664,4
47,103715156,214226569,3
59,10402,77819,3
...,...,...,...
8202,97167904,214778616,3
8216,98481320,124597967,9
8234,98793086,142710562,3
8242,99067259,108402614,3


## For each shopid, append all userids that have MAX order_counts userids onto the same line.
#### If multiple userids that have MAX order_counts, append together with "&" in between.

In [13]:
df_ob_max_appended = df_ob_max.groupby("shopid", sort = False)["userid"].unique().agg('&'.join).reset_index()
df_ob_max_appended.head(20)

,shopid,userid
0,100446829,2434757
1,10199219,8405753
2,10206302,95058664
3,103715156,214226569
4,10402,77819
5,104245736,188187242
6,104278513,80682958
7,104457555,214778616
8,104590058,81928284
9,104590579,213714377


#### Now we have the df for all shopids that have order brushed and the userids that brushed for them the most. Any other shopid not in the list are not deemed to have order brushed...

## Creating output

#### Create a new df with all the shopids (18870 of them) then merge with the appended userid df.

In [14]:
df_final = pd.DataFrame()
df_final["shopid"] = df["shopid"].unique()
df_final = df_final.merge(df_ob_max_appended, how = "left", on = "shopid").fillna("0")
df_final

,shopid,userid
0,6042309,0
1,104804492,0
2,8715449,0
3,190969466,0
4,2859407,0
...,...,...
18765,173249173,0
18766,212058195,0
18767,577768,0
18768,162561288,0


In [15]:
# Double checking: Those without order brushing
df_final[df_final["userid"] == "0"]

,shopid,userid
0,6042309,0
1,104804492,0
2,8715449,0
3,190969466,0
4,2859407,0
...,...,...
18765,173249173,0
18766,212058195,0
18767,577768,0
18768,162561288,0


In [16]:
# Double checking: Those with order brushing
df_final[df_final["userid"] != "0"]

,shopid,userid
20,68862371,67554410
48,27476241,174145893
52,192608876,213646699
107,10402,77819
133,114919380,117199298&9209456
...,...,...
17530,203070501,189834273
17957,156995951,204919399
17981,129113839,215408773
18410,123401085,12869645


#### Output to csv

In [17]:
df_final.to_csv("final.csv", index = False)